In [25]:
import requests
import json
import os
from time import sleep
API_KEY = {'X-API-Key': 'KZJ94OPT'}
session = requests.Session()
session.headers.update(API_KEY)

In [ ]:
class ApiException(Exception):
    pass

class HelperFunctions():
    
    def __init__(self):
        pass

    def ticker_bid_ask(self, session, ticker):
        payload = {'ticker': ticker}
        resp = session.get('http://localhost:9999/v1/securities/book', params=payload)  
        if resp.ok:
            book = resp.json()
            return book['bids'][0]['price'], book['asks'][0]['price']
        raise ApiException('Error getting bid / ask: The API key provided in this Python code must match that in the RIT client')

In [ ]:
array = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
sum = 0
sma_period = 12

for idx, i in reversed(list(enumerate(array))):
    if idx > len(array) - 13:
        sum += i

mov_avg = sum/sma_period
print(mov_avg)

# use this to determine whether to have more quantity / orders on buy side or sell side
# if price > sma -> buy more // vice versa

In [ ]:
resp = session.get('http://localhost:9999/v1/case')
ticker_history = session.get('http://localhost:9999/v1/securities/history', params={'ticker': 'ALGO'})

price = ticker_history.json()[0]['close']
print(price)

array = []
array.append(price)
sum = 0
sma_period = 12

for idx, i in reversed(list(enumerate(array))):
    if idx > len(array) - sma_period + 1:
        sum += i

mov_avg_num = sum/sma_period
print(mov_avg_num)

In [3]:
def get_order_book_stats(session, ticker, limit):

    orders = session.get('http://localhost:9999/v1/securities/book', params={'ticker': ticker, 'limit': limit})

    bids = orders.json()['bids']
    asks = orders.json()['asks']

    bid_cumulative_volume = 0
    bid_number_of_orders = 0

    ask_cumulative_volume = 0
    ask_number_of_orders = 0

    for i in bids:
        if i['trader_id'] == 'ANON':

            bid_cumulative_volume += i['quantity']
            bid_number_of_orders += 1

        # else:
        #     del bids[i]

    for i in asks:
        if i['trader_id'] == 'ANON':

            ask_cumulative_volume += i['quantity']
            ask_number_of_orders += 1

        # else:
        #     del asks[i]

    dict = {'Cumulative Vol Bid': bid_cumulative_volume,
            'Bid Num of Orders': bid_number_of_orders,
            'Cumulative Vol Ask': ask_cumulative_volume,
            'Ask Num of Orders': ask_number_of_orders,}

    return dict

    # for i in orders.json()['bid']:
    #     if orders.json()['bid']['trader-id'] != 'ANON':

print(get_order_book_stats(session, 'ALGO', 100))

ConnectionError: HTTPConnectionPool(host='localhost', port=9999): Max retries exceeded with url: /v1/securities/book?ticker=ALGO&limit=100 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001B35A893460>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [2]:
print(get_order_book_stats(session, 'ALGO', 100))

x = get_order_book_stats(session, 'ALGO', 100)

def calc_spread_cushion(order_books_stats):
    bid_vol = order_books_stats['Cumulative Vol Bid']
    ask_vol = order_books_stats['Cumulative Vol Ask']

    reduction_factor = 20
    price_cushion_factor = (bid_vol - ask_vol) / ask_vol / reduction_factor

    return price_cushion_factor

print(calc_spread_cushion(x))

NameError: name 'get_order_book_stats' is not defined

In [ ]:
def get_position(session, ticker):
    positions = session.get('http://localhost:9999/v1/securities', params={'ticker': ticker})
    position = positions.json()[0]['position']
    return position

print(get_position(session, 'ALGO'))

In [ ]:
def get_orders(session, status):
    payload = {'status': status}
    resp = session.get('http://localhost:9999/v1/orders', params=payload)
    if resp.status_code == 401:
        raise ApiException('The API key provided in this Python code must match that in the RIT client (please refer to the API hyperlink in the client toolbar and/or the RIT – User Guide – REST API Documentation.pdf)')
    orders = resp.json()
    # for i in orders
    return orders

print(get_orders(session, 'OPEN'))

In [1]:
list = [1,2,3]
len(list)

3

In [2]:
for i in range(5):
    print(i)

0
1
2
3
4


In [8]:
def get_orders(session, status):
    payload = {'status': status}
    resp = session.get('http://localhost:9999/v1/orders', params=payload)
    if resp.status_code == 401:
        raise ApiException('The API key provided in this Python code must match that in the RIT client (please refer to the API hyperlink in the client toolbar and/or the RIT – User Guide – REST API Documentation.pdf)')
    orders = resp.json()
    return orders

orders = get_orders(session, 'OPEN')


In [71]:
session.post('http://localhost:9999/v1/orders', params={'ticker': 'CNR', 'type': 'LIMIT', 'price': 18.00, 'quantity': 100, 'action': 'BUY'})
sleep(.5)
session.post('http://localhost:9999/v1/orders', params={'ticker': 'RY', 'type': 'LIMIT', 'price': 18.00, 'quantity': 100, 'action': 'BUY'})
sleep(.5)
session.post('http://localhost:9999/v1/orders', params={'ticker': 'AC', 'type': 'LIMIT', 'price': 18.00, 'quantity': 100, 'action': 'BUY'})
sleep(.5)

orders = session.get('http://localhost:9999/v1/orders', params={'status': 'OPEN'})
orders = orders.json()
print(orders)

[{'order_id': 11034, 'period': 1, 'tick': 140, 'trader_id': 'L', 'ticker': 'AC', 'quantity': 100.0, 'price': 18.0, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}, {'order_id': 10991, 'period': 1, 'tick': 140, 'trader_id': 'L', 'ticker': 'RY', 'quantity': 100.0, 'price': 18.0, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}, {'order_id': 10953, 'period': 1, 'tick': 139, 'trader_id': 'L', 'ticker': 'CNR', 'quantity': 100.0, 'price': 18.0, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}]


In [77]:
cnr_orders = []
ac_orders = []
ry_orders = []
order_index = 0

for i in orders:
    if orders[order_index]['ticker'] == 'AC':
        ac_orders.append(orders[order_index])
    if orders[order_index]['ticker'] == 'RY':
        ry_orders.append(orders[order_index])
    if orders[order_index]['ticker'] == 'CNR':
        cnr_orders.append(orders[order_index])
    order_index += 1

print(ac_orders)
print(ry_orders)
print(cnr_orders)

[{'order_id': 11034, 'period': 1, 'tick': 140, 'trader_id': 'L', 'ticker': 'AC', 'quantity': 100.0, 'price': 18.0, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}]
[{'order_id': 10991, 'period': 1, 'tick': 140, 'trader_id': 'L', 'ticker': 'RY', 'quantity': 100.0, 'price': 18.0, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}]
[{'order_id': 10953, 'period': 1, 'tick': 139, 'trader_id': 'L', 'ticker': 'CNR', 'quantity': 100.0, 'price': 18.0, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}]


In [57]:
 
for i in range(15):
    session.post('http://localhost:9999/v1/orders', params={'ticker': 'ALGO', 'type': 'LIMIT', 'price': 18.00, 'quantity': 100, 'action': 'BUY'})
    sleep(.25)

orders = get_orders(session, 'OPEN')
# print(orders)
order_len = len(orders)

if len(orders) >= 12:
                
    order_cancel_length = order_len - 12
    order_id_list = []

    for i in range(order_cancel_length):
        int_i = int(i)
        location = order_len - int_i - 1
        int(location)
        print(location)
        order_id = orders[location]['order_id']
        order_id_list.append(order_id)

    session.post('http://localhost:9999/v1/commands/cancel', params={'all': 0, 'ticker': 'ALGO', 'ids': order_id_list})    # {}'.format(order_idf))
                
                
    # print("this order has been cancelled")

44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12


In [64]:
session.post('http://localhost:9999/v1/orders', params={'ticker': 'CNR', 'type': 'MARKET', 'quantity': 9000, 'action': 'SELL'})

<Response [200]>

In [52]:
type(orders)
print(orders)

[{'order_id': 164, 'period': 1, 'tick': 22, 'trader_id': 'ahill3', 'ticker': 'ALGO', 'quantity': 100.0, 'price': 18.0, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}, {'order_id': 162, 'period': 1, 'tick': 22, 'trader_id': 'ahill3', 'ticker': 'ALGO', 'quantity': 100.0, 'price': 18.0, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}, {'order_id': 161, 'period': 1, 'tick': 21, 'trader_id': 'ahill3', 'ticker': 'ALGO', 'quantity': 100.0, 'price': 18.0, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}, {'order_id': 159, 'period': 1, 'tick': 21, 'trader_id': 'ahill3', 'ticker': 'ALGO', 'quantity': 100.0, 'price': 18.0, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}, {'order_id': 157, 'period': 1, 'tick': 21, 'trader_id': 'ahill3', 'ticker': 'ALGO', 'quantity': 100.0, 'price': 18.0, 'type': 'LIMIT', 'action': 'BUY', 'quantity_f